Code Libraries to Download

In [130]:
import requests
from time import sleep
from bs4 import BeautifulSoup as soup
from fpdf import FPDF
import unicodedata
import os

# Book Making


In [128]:
def clean_text(text):
    cleaned = ''.join(
        c for c in text
        if not unicodedata.category(c).startswith('C') and ord(c) <= 0xFFFF
    )
    return cleaned

In [137]:
# Config
book_name = 'Skyfire Avenue'
chapter_count = 905
book_limit = 15
font_path = 'Fonts/DejaVuSans.ttf'  # Make sure this exists

# Validate font path
if not os.path.exists(font_path):
    raise FileNotFoundError(f"Font not found at: {font_path}")

In [138]:
# PDF Setup
def new_pdf():
    pdf = FPDF()
    pdf.set_margins(left=10, top=20, right=10)
    pdf.add_font('DejaVu', '', font_path, uni=True)
    pdf.set_font('DejaVu', '', 12)
    pdf.add_page()
    return pdf

In [139]:
pdf = new_pdf()
chapter_written = False  # Flag to check if anything was written

for i in range(1, chapter_count + 1):
    sleep(1)
    try:
        url = f'https://freewebnovel.com/novel/skyfire-avenue/chapter-{i}'
        html_page = requests.get(url).content
        soup_page = soup(html_page, "html.parser")

        # Get chapter name
        title = soup_page.title
        chapter_name = title.string.split(' | ')[0].split(' - ')[1].strip()

        content_div = soup_page.find('div', {'id': 'article'})
        if not content_div:
            print(f"[Warning] No content in chapter {i}. Skipping.")
            continue
        paragraphs = content_div.find_all("p")
        if not paragraphs:
            print(f"[Warning] No paragraphs in chapter {i}. Skipping.")
            continue

        if i % book_limit == 1:
            pdf.set_font("DejaVu", '', 28)
            pdf.multi_cell(0, 10, txt=book_name, align='C')
            pdf.ln(10)

        pdf.set_font("DejaVu", '', 18)
        pdf.multi_cell(0, 10, txt=clean_text(chapter_name))
        pdf.ln(5)

        pdf.set_font("DejaVu", '', 12)
        for j, p in enumerate(paragraphs):
            if j == 0:
                continue
            text = p.text.strip()
            if not text:
                continue
            if any(ord(c) > 0xFFFF for c in text):
                print(f"[Info] High Unicode in chapter {i}, paragraph {j}")
            text = clean_text(text)
            try:
                pdf.multi_cell(0, 8, txt=text)
                chapter_written = True
            except Exception as e:
                print(f"[Error] Writing paragraph {j} in chapter {i}: {e}")

        pdf.ln(10)

        if (i % book_limit == 0 or i == chapter_count) and chapter_written:
            start = i - book_limit + 1 if i >= book_limit else 1
            filename = f"Skyfire Avenue/{book_name} Chapters {start} - {i}.pdf"
            print(f"[Output] Writing {filename}")
            pdf.output(filename)
            pdf = new_pdf()
            chapter_written = False

    except Exception as e:
        print(f"[Error] Failed at chapter {i}: {e}")

[Output] Writing Skyfire Avenue/Skyfire Avenue Chapters 1 - 15.pdf
[Output] Writing Skyfire Avenue/Skyfire Avenue Chapters 16 - 30.pdf
[Info] High Unicode in chapter 33, paragraph 45
[Info] High Unicode in chapter 34, paragraph 21
[Info] High Unicode in chapter 43, paragraph 27
[Info] High Unicode in chapter 44, paragraph 11
[Output] Writing Skyfire Avenue/Skyfire Avenue Chapters 31 - 45.pdf
[Info] High Unicode in chapter 59, paragraph 43
[Output] Writing Skyfire Avenue/Skyfire Avenue Chapters 46 - 60.pdf
[Info] High Unicode in chapter 73, paragraph 30
[Output] Writing Skyfire Avenue/Skyfire Avenue Chapters 61 - 75.pdf
[Output] Writing Skyfire Avenue/Skyfire Avenue Chapters 76 - 90.pdf
[Info] High Unicode in chapter 94, paragraph 45
[Info] High Unicode in chapter 95, paragraph 4
[Output] Writing Skyfire Avenue/Skyfire Avenue Chapters 91 - 105.pdf
[Output] Writing Skyfire Avenue/Skyfire Avenue Chapters 106 - 120.pdf
[Output] Writing Skyfire Avenue/Skyfire Avenue Chapters 121 - 135.pdf
[